# Prediction Model for California Real Estate Values

    Adapted from Data Science book used in Practical Data Analysis class


# Location to save the pickled trained model


You need to provide the full path to the pickled file, or else TabPy will fail to load it. 
Change the C:\\Code\\Python path as you wish
#joblib.dump(final_model2, "C:\\Code\\Python\\CA_real_estate_model2.pkl")


# Get the Data
   Template code to download compressed file, uncompress and save to local system

In [ ]:
import os # used just to create the folders on current directory
import tarfile
from six.moves import urllib # Note needed for Python3 -- compatibility with Python2

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data() # end result is housing.csv file imported to Jupyter local directory


In [ ]:
# Check the version of Scikit-learn

#import nltk
import sklearn

#print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))


# The scikit-learn version used for this demo is 0.22.1

# Load df "housing"
Contains the original data from csv file

In [ ]:
import pandas as pd
import numpy as np

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()


# Goals for this exercise
    1) Predict the value of median_house_value attribute (target label)
    2) We know median_income is a strong predictor of median_household_value. We will create the training set stratified by median_income
    3) Input attributes total_rooms, total_bedrooms and population are SUM by district and therefore useless to draw any correlations. We know this is needed by visually inspecting the source data.
    We will derive average values by dividing those attributes by the number of households
    4) Categorical attribute ocean_proximity will be changed to numeric using OneHotEncoding
    5) Missing values will be replaced with the average for the column

# Stratify the split by Median Income range
    Because Median Income is an important indicator of house values
    We want to ensure all median income ranges are represented in the training set
    We don't have a range indicator in the dataset, so we will create a function to classify the median income into ranges, then run the split. We will drop the range indicator when we are done with the split.

In [ ]:
#Create a category for median income ranges. Limit it to 5.

# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
da=housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)


# Split dataset into training and test sets
    Use sklearn to perform stratified split by Median Income ranges

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# We don't need "income_cat" anymore. Drop it from split sets
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# Save the test set to CSV, so we cn reuse it in Tableau 
strat_test_set.to_csv("c:\Code\Python\housing_test.csv")

In [ ]:
# Save the training set to CSV, so we cn reuse it in Tableau 
strat_train_set.to_csv("c:\Code\Python\housing_training.csv")

# Prepare training set for ML algorithm

# Step 1 - Split input attributes and target(label)
    Drop median_house_value from the training set
    
    Data frame "housing" is now of INPUT ATTRIBUTES only! 
    Data frame housing_num contains on numeric attributes --> only used to grab numeric column names -- is there a smarter way?
    Array housing_labels is the saved copy of target labels

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels from training set
housing_num = housing.drop('ocean_proximity', axis=1) # only the numeric values
housing_labels = strat_train_set["median_house_value"].copy() # save labels from training set for later use

# Step 2 - Define sklearn transformation functions

Define a custom function to create derived columns rooms_per_household and population_per_household
Nulls and categories will be handled with standard sklearn transformation classes

In [ ]:
from sklearn.preprocessing import FunctionTransformer

# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

#add_extra_features is your custom function
def add_extra_features(X, add_bedrooms_per_room=True):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]  # concatenate new columns to existing columns in X
    else:
        return np.c_[X, rooms_per_household, population_per_household]


# Step 3 - Apply all 4 transformations in a Pipeline
    imputer = Handle nulls - replace all nulls values with the average value of the column
    std_scaler = apply default feature scaler to all numbers
    attribs_adder = custom function that creates derived columns rooms, bedrooms and population per household
    OneHotEncoder = changes category "ocean_proximity" to numeric values

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler # to scale numbers
from sklearn.impute import SimpleImputer # to handle nulls
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing_num)   # List of numeric columns
cat_attribs = ["ocean_proximity"] # List of categorical attributes

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),   # apply numeric pipeline to numeric attributes
        ("cat", OneHotEncoder(), cat_attribs),# apply category pipeline to categories
    ])

# housing is the training data without the labels
# It's not really necessary to separate categorical from numerical attributes, the ColumnTransformer applies
#    functions to the selected numeric or category columns

housing_prepared = full_pipeline.fit_transform(housing)



# Sanity check

In [ ]:
df_housing_prepared = pd.DataFrame(
    housing_prepared,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household", "cat1", "cat2", "cat3", "cat4", "cat5"])
# Check that all columns look good with their labels
df_housing_prepared.head()

In [ ]:
housing_labels

In [ ]:
housing

# Select and train a model

# First Model: Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

# Compare predictions to actual values
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

# Evaluate how good the model is 
    1) How does the model fit my training data? 
    2) How does the model fit another dataset - not the training data
    
    If the model does not fir the training data, then the model is not good - for example, this data does not fit a linear function, so using linear regression will not produce good predictions

# Calculate mean_squared_error and mean_absolute_error
    In a Linear regression, our objective is to minimize the MSE. This translates to calculating points on the line of best fit that are closest to the actual points on the graph
    
    We will calculate some statistical error mesasures comparing the predicted values to the known labels
    
    
    
    

# Meaning of RMSE
As the square root of a variance, RMSE can be interpreted as the standard deviation of the unexplained variance,
and has the useful property of being in the ***same units as the response variable. 

Lower values of RMSE indicate better fit.

RMSE is a good measure of how accurately the model predicts the response, and it is the most important criterion 
for fit if the main purpose of the model is prediction.

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)  # run prediction on entire training dataset
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
lin_mae

Okay,  this  is  better  than  nothing  but  clearly  not  a  great  score:  most  districts’
median_housing_values  range between $120,000 and $265,000, so a typical prediction error of $68,628 is not very satisfying.
(See 25th and 75th percentiles below)

In [ ]:
housing_labels.describe()

# Coeficient and Intercept

In [ ]:
print (df_housing_prepared.columns)
print ('Coeficients: \n,', lin_reg.coef_)
print ('Intercept: \n', lin_reg.intercept_)

for i in range(len(df_housing_prepared.columns)):
    print ('Coefficient of ', df_housing_prepared.columns[i], ' = ', lin_reg.coef_[i])
           

# Conclusion - error too large - UNDERFITTING
    The Linear Regression model is off by $49k (MAE) or $68k (MSE)
    You model does not even fir the training dataset

In [ ]:
housing_prepared.shape

In [ ]:
housing_labels.shape

In [ ]:
housing_predictions.shape

In [ ]:
import matplotlib.pyplot as  plt
plt.scatter(df_housing_prepared['population_per_household'], housing_labels, color='red')
plt.plot(df_housing_prepared['population_per_household'], housing_predictions, color='blue')
plt.title='Population per Houselhold  vs House Price'
plt.xlabel='Population per Household'
plt.ylabel='House Price'

# Second Model: Decision Tree Regressor
A decision tree is a supervised machine learning model used to predict a target by learning decision rules from features. As the name suggests, we can think of this model as breaking down our data by making a decision based on asking a series of questions

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

# Model metric: RMSE (Root Mean Squared Error)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

# RMSE = 0 Model not good either = OVERFITTING
    Means that every predicted value is exaclty the same as the real value. 
    You model fits the training dataset TOO WELL ==> it cannot generlize and make predictions on values it never saw before
    
    If the model does not capture the dominant trend that we can all see (positively increasing, in our case), it can’t predict a likely output for an input that it has never seen before — defying the purpose of Machine Learning to begin with!

In [ ]:
# Show how model did not make any generalizations. All predicted points are on top of target values
import matplotlib.pyplot as  plt
plt.scatter(df_housing_prepared['population_per_household'], housing_labels, color='red')
plt.plot(df_housing_prepared['population_per_household'], housing_predictions, color='blue')
plt.title='Population per Houselhold  vs House Price'
plt.xlabel='Population per Household'
plt.ylabel='House Price'

# Random Forest
max_depth refers to the maximum depth of the tree and n_estimators, the number of trees in the forest. Ideally, you can expect a better performance from your model when there are more trees.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

median_housing_values  range between $120,000 and $265,000
Error of 22k (about 18%) is acceptable

# Example using GridSearch to tune Hyperparameters

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler


# Perform Grid-Search
gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(3,7),
        'n_estimators': (10, 50, 100, 1000),
        },
    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
grid_result = gsc.fit(housing_prepared, housing_labels)
best_params = grid_result.best_params_
    


In [ ]:
grid_result

In [ ]:
best_params

In [ ]:
rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],random_state=False, verbose=False)
# Perform K-Fold CV
scores = cross_val_score(rfr, housing_prepared, housing_labels, cv=10, scoring='neg_mean_absolute_error')



In [ ]:
scores

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest2_reg = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],random_state=False, verbose=False)
forest2_reg.fit(housing_prepared, housing_labels)

housing_predictions = forest2_reg.predict(housing_prepared)
forest2_mse = mean_squared_error(housing_labels, housing_predictions)
forest2_rmse = np.sqrt(forest2_mse)
forest2_rmse

Running RandomForest with 100 estimators made the fit to the data set worse

# K-fold Cross Validation
    Cross-validation is a statistical method used to estimate the skill of machine learning models.Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample.
    
    Three common tactics for choosing a value for k are as follows:

Representative: The value for k is chosen such that each train/test group of data samples is large enough to be statistically representative of the broader dataset.

k=10: The value for k is fixed to 10, a value that has been found through experimentation to generally result in a model skill estimate with low bias a modest variance.

k=n: The value for k is fixed to n, where n is the size of the dataset to give each test sample an opportunity to be used in the hold out dataset. This approach is called leave-one-out cross-validation.
    

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())



In [ ]:
tree_scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)
print ('MSE scores')
display_scores(tree_scores)
print('')
print ('RMSE scores')
display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

# We already know the tree and liner models don't fit the training data. We applied the cross val for education purposes

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
print ('Model fit to the training set')
print ('-----------------------------')
print ('Linear regression RMSE = ', lin_rmse)
print ('Decision Tree RMSE = ', tree_rmse)
print ('Random Forest RMSE = ', forest_rmse)


print ('')
print ('Model fit based on 10-Fold Validation')
print ('-------------------------------------')
print ('Linear Regression Mean RMSE Scores = ', lin_rmse_scores.mean(), ' ~ STD: ', lin_rmse_scores.std())
print ('Decision Tree RMSE Scores = ', tree_rmse_scores.mean(), ' ~ STD: ', tree_rmse_scores.std())
print ('Random Forest RMSE Scores = ', forest_rmse_scores.mean(), ' ~ STD: ', forest_rmse_scores.std())



# Fine tune your models
    Look for the best combination of hyperparameters for your selected model (RandomForest)

# Method 1: Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
   ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels) # Best model is stored in grid_search variable

The best hyperparameter combination found:

In [ ]:
grid_search.best_estimator_

# Method 2: Randomized Search
    Use this when you don't know the specific values you want to try. This uses a large range with min/max values for estimators and features
    n_iter=10 = will try 10 combinations of randomly selected parameters

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }
# n_iter=10 = will try 10 combinations of randomly selected parameters
forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

# Apply to test data


In [ ]:
grid_search.best_estimator_

In [ ]:
rnd_search.best_estimator_

# Testing the model
    We need the saved TESTING set we split out of the original dataset earlier: strat_test_set
    

In [ ]:
strat_test_set.head(10)

In [ ]:
final_model1 = grid_search.best_estimator_
final_model2 = rnd_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions1 = final_model1.predict(X_test_prepared)
final_predictions2 = final_model2.predict(X_test_prepared)

final_mse1 = mean_squared_error(y_test, final_predictions1)
final_rmse1 = np.sqrt(final_mse1)
print(final_rmse1)

final_mse2 = mean_squared_error(y_test, final_predictions2)
final_rmse2 = np.sqrt(final_mse2)
print(final_rmse2)

In [ ]:
from sklearn.externals import joblib

# Change the path to the pickled file to your preferred location
# If a path is not provided, Jupyter saves the file to the same folder Jupyter was launched from (in my case, C:\Code\Python)
# Witout a full path, the TabPy server will not be able to locate and load the picked model! 

joblib.dump(final_model2, "C:\\Code\\Python\\CA_real_estate_model2.pkl")


# Create TabPy endpoint for selected model
    The input for the model must match the format of the training dataset used to create and train the model. 
    In the model creation, several transformations were applied to the data, and that code must be applied in '
    the TabPy endpoint as well. 
    
    In Tableau, the workbook will connect to an Excel file containing data from the test dataset created in this Jupyter notebook, in exactly the same layout. Tableau will then call the PredctHomeValue function via the "RealEstateDemo" endpoint to obtain the predicted real estate values. 

In [3]:

def PredictHomeValue(longitude,latitude,housing_median_age,total_rooms,total_bedrooms,
                     population,households,median_income,ocean_proximity):
    # Load arguments to pandas dataframe
    import pandas as pd
    import numpy as np

    housing = pd.DataFrame({
     "longitude": longitude, "latitude": latitude, "housing_median_age": housing_median_age,"total_rooms": total_rooms,
     "total_bedrooms": total_bedrooms,"population": population,"households": households,"median_income": median_income,
      "ocean_proximity": ocean_proximity
    })
    
   # Run the test data through pipeline to cleanse the datataset 

    from sklearn.preprocessing import FunctionTransformer
    housing_num = housing.drop('ocean_proximity', axis=1) # only the numeric values

    # get the right column indices: safer than hard-coding indices 3, 4, 5, 6
    rooms_ix, bedrooms_ix, population_ix, household_ix = [
        list(housing.columns).index(col)
        for col in ("total_rooms", "total_bedrooms", "population", "households")] 
 
    #add_extra_features is your custom function
    def add_extra_features(X, add_bedrooms_per_room=True):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]  # concatenate new columns to existing columns in X
        else:
            return np.c_[X, rooms_per_household, population_per_household]
    
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler # to scale numbers
    from sklearn.impute import SimpleImputer # to handle nulls
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import OneHotEncoder

    num_attribs = list(housing_num)   # List of numeric columns
    cat_attribs = ["ocean_proximity"] # List of categorical attributes

    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
        ])

    full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),   # apply numeric pipeline to numeric attributes
        ("cat", OneHotEncoder(), cat_attribs),# apply category pipeline to categories
        ])

    housing_prepared = full_pipeline.fit_transform(housing)

    # Load previously saved model and use it for predictions
    # Use the full path where picked file is saved. Jupyter finds the model on the same folder it was saved
    # TabPy needs the full reference to locate the file 
    
    from sklearn.externals import joblib
    my_model_loaded = joblib.load("C:\\Code\\Python\\CA_real_estate_model2.pkl") 

    final_predictions = my_model_loaded.predict(housing_prepared)

    return list(final_predictions)



In [4]:
PredictHomeValue(_arg1,_arg2,_arg3,_arg4,_arg5,_arg6,_arg7,_arg8,_arg9)

[446339.27777777775,
 385031.9666666667,
 308553.4611111111,
 204964.49444444446,
 166965.58333333334,
 180360.5888888889,
 279915.1722222222,
 391298.52777777775,
 108102.78888888888]

In [5]:
from tabpy.tabpy_tools.client import Client

connection = Client('http://localhost:9004/')
# Publish the  function to TabPy server 
connection.deploy('RealEstateDemo',
                  PredictHomeValue,
                  'Prediction Model for Property Values',
                 override = True)

## End of jupyter notebook